In [1]:
import pandas as pd
import numpy as np
from azureml.core import Workspace, Datastore, Dataset
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, TimeSeriesSplit,RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_percentage_error
from sklearn.tree import DecisionTreeRegressor
import xgboost as xgb
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')
import lightgbm as lgb
import pickle
import joblib
from sklearn.ensemble import StackingRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder, StandardScaler, OneHotEncoder ,FunctionTransformer
import xgboost as xgb
from scipy.special import inv_boxcox
 
from sklearn.feature_selection import mutual_info_regression
from sklearn.impute import SimpleImputer
from scipy import stats
import sklearn
 

Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.scriptrun = azureml.core.script_run:ScriptRun._from_run_dto with exception (cryptography 42.0.7 (/Users/vallirajasekar/opt/anaconda3/lib/python3.9/site-packages), Requirement.parse('cryptography<4.0.0,>=3.3.1; extra == "crypto"'), {'PyJWT'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.PipelineRun = azureml.pipeline.core.run:PipelineRun._from_dto with exception (cryptography 42.0.7 (/Users/vallirajasekar/opt/anaconda3/lib/python3.9/site-packages), Requirement.parse('cryptography<4.0.0,>=3.3.1; extra == "crypto"'), {'PyJWT'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.ReusedStepRun = azureml.pipeline.core.run:StepRun._from_reused_dto with exception (cryptography 42.0.7 (/Users/vallirajasekar/opt/anaconda3/lib/python3.9/site-packages), Requirement.parse('cryptography<4.0.0,>=3.3.1; extra == "crypto"'), {'PyJWT'}).
Failure wh

OSError: dlopen(/Users/vallirajasekar/opt/anaconda3/lib/python3.9/site-packages/lightgbm/lib/lib_lightgbm.so, 0x0006): Library not loaded: /usr/local/opt/libomp/lib/libomp.dylib
  Referenced from: <C3EB28DD-60B6-3334-AFA2-72BBBF9DBAEF> /Users/vallirajasekar/opt/anaconda3/lib/python3.9/site-packages/lightgbm/lib/lib_lightgbm.so
  Reason: tried: '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/usr/local/lib/libomp.dylib' (no such file), '/usr/lib/libomp.dylib' (no such file, not in dyld cache)

In [ ]:
# Initialize Workspace and load data
ws1 = Workspace.from_config()  # Connect to Azure ML workspace using config file
datastore = Datastore.get(ws1, "azsgcsschecstr03d_datastore")  # Retrieve datastore by name
dataset = Dataset.Tabular.from_parquet_files(path=(datastore, '/S4/PROCESSED_ZONE/SALES_FORECAST/HISTORICAL_SALES_DATA/*'))  # Load parquet files into a dataset
df = dataset.to_pandas_dataframe()  # Convert dataset to a pandas DataFrame

In [ ]:
df.groupby('COUNTRY_CODE')['SALES_VALUE_USD'].sum().reset_index().sort_values('SALES_VALUE_USD',ascending=False)
 
df_hk = df.query('COUNTRY_CODE=="TH"')
 
columns_to_drop = ['BUSINESS_SEGMENT', 'TOTAL_SALES_QTY', 'BUSINESS_LINE_DESC', 'ORD_CURRENCY',
                   'BUSINESS_SEGMENT_DESC', 'BUSINESS_MODEL_1_DESC', 'ATC_3_ID', 'ATC_4_ID', 'ATC_2_DESC', 'BRAND_CODE',
                    'BILLING_DATE', 'CLIENT_DESC', 'CUST_GRP_SALES_ORDER', 'CHANNEL_1', 'CHANNEL_2', 'CHANNEL_3']
df_hk.drop(columns=columns_to_drop, axis=1, inplace=True)  # Drop irrelevant columns
 
df_hk_gp = df_hk.groupby(['COUNTRY_CODE','MATERIAL_GRP4_CODE','ATC_2_ID','MATERIAL_CODE','BILLING_YEAR','BILLING_MONTH']).agg({
 
    'UNIT_PRICE': 'first',
   
    'LOST_SALES_VALUE_USD': 'sum',
 
    'TOTAL_FOC_SALES_QTY': 'sum',
 
    'SALES_VALUE_USD': 'sum',
 
    'NET_SALES_QTY':'sum'
 
}).reset_index()
 
 
 

In [ ]:
df_hk_gp['ts_key'] = df_hk_gp['COUNTRY_CODE']+'_'+df_hk_gp['MATERIAL_GRP4_CODE']+'_'+df_hk_gp['ATC_2_ID']+'_'+df_hk_gp['MATERIAL_CODE']
 
df_hk_gp['BILLING_MONTH'] = df_hk_gp['BILLING_MONTH'].str.zfill(2)
 
 
df_hk_gp['fiscal_month'] = df_hk_gp['BILLING_YEAR']+df_hk_gp['BILLING_MONTH']
 
df_in = df_hk_gp.copy()
 
df_in.drop(['BILLING_MONTH','BILLING_YEAR'],axis=1,inplace=True)
 
nz_df=df_in[df_in['SALES_VALUE_USD']>0]
l1=nz_df.groupby('ts_key').filter(lambda x:len(x)>36)['ts_key'].unique()
df_new=df_in[df_in.ts_key.isin(l1)]
 
def pad_time_series(df_in:pd.DataFrame) -> pd.DataFrame:
    min_month = df_in['fiscal_month'].min()
    max_month = df_in['fiscal_month'].max()
    fiscal_months = pd.date_range(start=min_month[:6]+"01",end=max_month[:6]+"01",freq='MS',).strftime('%Y%m').tolist()
    unique_ts_key = df_in['ts_key'].unique()
    result = pd.DataFrame()
    for ts_key in unique_ts_key:
        subset = df_in[df_in['ts_key']==ts_key]
        COUNTRY_CODE=subset.iloc[0]['COUNTRY_CODE']
        MATERIAL_GRP4_CODE = subset.iloc[0]['MATERIAL_GRP4_CODE']
        MATERIAL_CODE = subset.iloc[0]['MATERIAL_CODE']
        full_index=pd.DataFrame({'fiscal_month':fiscal_months,'MATERIAL_GRP4_CODE':MATERIAL_GRP4_CODE,'MATERIAL_CODE':MATERIAL_CODE,'COUNTRY_CODE':COUNTRY_CODE,
        "ts_key":ts_key
        })
 
        merged_subset = pd.merge(full_index,subset,on=['COUNTRY_CODE','MATERIAL_GRP4_CODE','MATERIAL_CODE','ts_key','fiscal_month'],how='left')
        merged_subset['NET_SALES_QTY'].fillna(0,inplace=True)
        merged_subset['SALES_VALUE_USD'].fillna(0,inplace=True)
        result = pd.concat([result,merged_subset],ignore_index=True)
    return result
 
 
 
 
df_in['year'] = df_in['fiscal_month'].str[:4]
df_in['month'] = df_in['fiscal_month'].str[4:]
df_in['fiscal_month'] = df_in['year'].astype('str')+"0"+df_in['month'].astype('str')
 
def fiscal_month_of_year(x):
    try:
        return int(x)%1000
    except:
        return None
 
def fiscal_month_from_iso_week(iso_week :int):
    year = iso_week//100
    week=iso_week%100
    month= month_455_from_iso_week(week)
    return int(year * 1000 + month)
 
def month_455_from_iso_week(iso_week:int):
    iso_week_zero_indexed = iso_week -1
    months_elapsed = (iso_week_zero_indexed //13) * 3
    week_in_quarter = iso_week_zero_indexed % 13
    if week_in_quarter <=3:
        return min(months_elapsed + 1,12)
    elif week_in_quarter <=7:
        return min(months_elapsed + 2, 12)
    else:
        return min(months_elapsed + 3, 12)
 
def generate_fiscal_calendar( year:int):
    iso_week_in_year = date(year, 12, 28).isocalendar()[1]
    iso_weeks = [year * 100 + w for w in range ( 1, iso_week_in_year +1 )]
    dates = [date_from_calweek(iw) for iw in iso_weeks ]
    fiscal_months = [fiscal_month_from_iso_week(iw) for iw in iso_weeks]
   
    df = pd.DataFrame({'date_t':dates,
    'fiscal_month': fiscal_months})
 
    df_month = (df.groupby('fiscal_month')['date_t'].agg(lambda x: min(x)).reset_index().rename(columns={'date_t':'start_month'}))
    df_month['end_month'] =  df.groupby('fiscal_month')['date_t'].agg(lambda x: max(x))
    df_month['end_month'] = pd.to_datetime(df_month['end_month']) + pd.DateOffset(days=6)
    return df_month
   
from datetime import date, datetime
 
def date_from_calweek(calweek : int):
    iso_year = calweek //100
    iso_weeknumber = calweek % 100
    iso_weekday = 1
    return datetime.strptime(
    '{:04d} {:02d} {:d}'.format(iso_year, iso_weeknumber, iso_weekday), '%G %V %u').date()
   
 
def fiscal_month(iso_week:int):
    year = int(str(iso_week)[:4])
    week =  int(str(iso_week)[:-2])
    month = month_455_from_iso_week(week)
    return int( year *1000 + month)
 
def generate_all_relevant_fiscal_months(current_fiscal_month, n_horizon:int):
    current_year = current_fiscal_month // 1000
    n_years = n_horizon //12
    all_year_required = [current_year + y  for y in range(n_years + 2)]
    df_months = pd.concat([generate_fiscal_calendar(y)
     for y in all_year_required], ignore_index=True)
 
    ix_current_month =  df_months.query(f'fiscal_month=={current_fiscal_month}').index[0]
    result = df_months.iloc[ix_current_month:ix_current_month + n_horizon]
    assert len(result) > 0, 'results found'
    return result.reset_index(drop=True)
 
 
def seasonal_naive(row):
    return row['lag_12']
 
def create_pipeline_for_model_rf():
    # List of all categorical columns
    categorical_columns = ['month_of_year', 'ATC_2_ID', 'REGION','MATERIAL_CODE']
    # Specify categories for OrdinalEncoder
    categories = [
        [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],  # for 'month_of_year'
        'auto',  # for 'ATC_2_ID'
        'auto',   # for 'Region'
        'auto'
    ]
    # Create an OrdinalEncoder instance
    ordinal_encoder = OrdinalEncoder(
        categories=categories,
        handle_unknown='use_encoded_value',
        unknown_value=-1
    )
    # Initialize the RandomForestRegressor
    rf_model = RandomForestRegressor(n_jobs=6, n_estimators=100)
    # Create a pipeline with ColumnTransformer and RandomForestRegressor
    model = Pipeline([
        ('col_trans', ColumnTransformer(
            transformers=[('categorical', ordinal_encoder, categorical_columns)],
            remainder="passthrough")),
        ('scaler', StandardScaler()),
        ('rf', rf_model)
    ])
    return model
 


 
def create_pipeline_for_model_rf():
    categorical_columns=['month_of_year']
    categories=[[1,2,3,4,5,6,7,8,9,10,11,12]]
    ordinal_encoder = OrdinalEncoder(categories=categories,
    handle_unknown='use_encoded_value',
    unknown_value=-1)
    rf_model = RandomForestRegressor(n_jobs=6,n_estimators=200,max_depth=4,min_samples_split=4)#,n_estimators=120,max_depth=4,min_samples_split=4,max_features='log2')
    model = Pipeline([
       ( 'col_trans', ColumnTransformer(transformers=[('categorical',ordinal_encoder,categorical_columns)], remainder="passthrough")),
    ('scaler',StandardScaler()),
    ('rf',rf_model)
    ])
 
    return model
 
 
 
def create_pipeline_for_model_xgb():
    categorical_columns=['month_of_year']
    categories=[[1,2,3,4,5,6,7,8,9,10,11,12]]
    ordinal_encoder = OrdinalEncoder(categories=categories,
    handle_unknown='use_encoded_value',
    unknown_value=-1)
    xgboost = xgb.XGBRegressor(n_estimators=100,learning_rate=0.01,max_depth=2,booster='gbtree',device='cuda',tree_method='hist')
    model = Pipeline([
       ( 'col_trans', ColumnTransformer(transformers=[('categorical',ordinal_encoder,categorical_columns)], remainder="passthrough")),
    ('scaler',StandardScaler()),
    ('xgb',xgboost)
    ])
 
    return model
 
def create_pipeline_for_model_lgbm():
    categorical_columns = ['month_of_year']
    categories = [[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]]
    ordinal_encoder = OrdinalEncoder(
        categories=categories,
        handle_unknown='use_encoded_value',
        unknown_value=-1
    )
    lightgbm = lgb.LGBMRegressor(
        n_estimators=100,
        learning_rate=0.01,
        max_depth=2,
        boosting_type='goss',
        num_leaves=30
       
    )
    model = Pipeline([
        ('col_trans', ColumnTransformer(
            transformers=[('categorical', ordinal_encoder, categorical_columns)],
            remainder='passthrough'
        )),
        ('scaler', StandardScaler()),
        ('lgbm', lightgbm)
    ])
    return model
 
 
def create_features_and_labels_per_horizon(df_ts_key,horizons=None, target='SALES_VALUE_USD'):
    #This method considers the target and create multiple features and returns a dataframe with features at multiple horizons
    df_out = df_ts_key.copy()
 
    lag_window_size= 12
    long_window_size = 24
 
    if horizons is None:
        horizons = [0,1,2,3]
    df_out['month'] = df_out.month.astype('int64')
    df_out['year'] =  df_out.year.astype('int64')
    df_out['quarter'] =  (df_out['month'] -1) //3 + 1
    df_out['quarter'] = df_out.quarter.astype('int64')
    df_out['moving_avg_last_quarter'] = df_out[target].rolling(3).mean(3)
 
    for lag in range(1,13):
        df_out[f'lag_{lag}_at_submission_month'] = df_out[target].shift(lag)
        df_out[f'lag_{lag}_last_year'] = df_out[target].shift( lag * 12)
 
    for lag in range(1, long_window_size + 1):
        df_out[f'lag_{lag}'] = df_out[target].shift(lag)
   
    for lag in range(1 , long_window_size + 1):
        df_out[f'lag_{lag}_moving_avg'] = df_out[f'lag_{lag}'].rolling(window=long_window_size).mean()
   
    for lag in range(1, 25):
        df_out[f'moving_avg_{lag}_months'] = df_out[target].rolling(window = lag * 12).mean()
   
    for span in [5,10,20]:
        df_out[f'ema_{span}'] = df_out[target].ewm(span=span, adjust=False).mean()
   
    df_out['YoY_growth'] = df_out[target].shift(12)
    df_out['Yoy_percentage_growth'] = (df_out[target] / df_out[target].shift(12) - 1) * 100
 
    df_out['quarter_growth'] = df_out[target].shift(3)
    df_out['QoQ_percentage_growth'] = (df_out[target] / df_out[target].shift(3) - 1) * 100
 
    df_out['sin_month'] = np.sin(2 * np.pi * (df_out['month'] -1) /3 )
    df_out['cos_month'] = np.cos(2 * np.pi * (df_out['month'] -1) /3 )
 
    df_out['diff_at_submission_month'] = df_out[target].shift(1) - df_out[target].shift(2)
    df_out['ema_growth'] = df_out['ema_20'].diff()
 
    dfs_per_h=[]
 
    for h in horizons:
        df_h = df_out.copy()
        df_h['submission_month'] = df_out['fiscal_month']
        df_h['fiscal_month'] = df_out['fiscal_month'].shift(-h)
        df_h['horizon'] = h
        df_h['lag_12'] = df_out[target].shift(12 - h)
        df_h['month_of_year'] = [fiscal_month_of_year(x) for x in df_out['fiscal_month'].shift(-h)]
        df_h['y'] = df_out[target].shift(-h)
 
        dfs_per_h.append(df_h)
 
    return pd.concat(dfs_per_h, ignore_index=True)
 
   
 
 
 
 
def create_stacking_pipeline_rf_xgb():
    rf_pipeline = create_pipeline_for_model_rf()
    xgb_pipeline = create_pipeline_for_model_xgb()
 
    estimators = [
        ('rf', rf_pipeline),
        ('xgb', xgb_pipeline)
    ]
 
   
 
    stacking_regressor = StackingRegressor(
        estimators=estimators,
        final_estimator=DecisionTreeRegressor(),
        passthrough=True  
    )
 
    return stacking_regressor
 
def mi_feature_selection(df_in: pd.DataFrame, features: list, sales_col='clipped_orders', threshold=1.15) -> list:
    """
    Perform mutual information feature selection with improved error handling and data preprocessing.
    Parameters:
    - df_in: Input DataFrame.
    - features: List of feature columns to evaluate.
    - sales_col: The target column for sales data.
    - threshold: Minimum mutual information score for feature selection.
    Returns:
    - List of selected features with mutual information score greater than the threshold.
    """
    df_out = df_in.copy()
    # Check if all specified features are in the DataFrame
    available_features = [f for f in features if f in df_out.columns]
    missing_features = set(features) - set(available_features)
    if missing_features:
        print(f"Warning: The following features are not in the DataFrame: {missing_features}")
    if sales_col not in df_out.columns:
        raise ValueError(f"Sales column '{sales_col}' not found in the DataFrame.")
    # Ensure sales_col is not in the feature list
    if sales_col in available_features:
        available_features.remove(sales_col)
    # Debug: Print shapes and column counts
    #print(f"Shape of df_out: {df_out.shape}")
    #print(f"Number of available features: {len(available_features)}")
    #print(f"Available features: {available_features}")
    # Select only non-null columns for imputation
    non_null_features = [f for f in available_features if df_out[f].notna().any()]
    #print(f"Number of non-null features: {len(non_null_features)}")
    #print(f"Non-null features: {non_null_features}")
    # Debug: Check for NaN and infinite values before processing
    nan_count = df_out[non_null_features + [sales_col]].isna().sum()
    inf_count = np.isinf(df_out[non_null_features + [sales_col]]).sum()
    #print("NaN count per column:", nan_count)
    #print("Infinite count per column:", inf_count)
    # Replace infinite values with NaN
    df_out = df_out.replace([np.inf, -np.inf], np.nan)
    # Impute missing values
    imputer = SimpleImputer(strategy='mean')
    try:
        imputed_data = imputer.fit_transform(df_out[non_null_features + [sales_col]])
        df_imputed = pd.DataFrame(imputed_data,
                                  columns=non_null_features + [sales_col],
                                  index=df_out.index)
    except ValueError as e:
        print(f"Error during imputation: {e}")
        raise
    # Normalize features
    scaler = StandardScaler()
    df_scaled = pd.DataFrame(scaler.fit_transform(df_imputed[non_null_features]),
                             columns=non_null_features,
                             index=df_imputed.index)
    # Keep the sales column unscaled
    df_scaled[sales_col] = df_imputed[sales_col]
    # Calculate mutual information scores
    mi_scores = mutual_info_regression(df_scaled[non_null_features], df_scaled[sales_col])
    # Create a DataFrame from the mutual information scores
    mi_score_df = pd.DataFrame({'feature': non_null_features, 'score': mi_scores})
    mi_score_df = mi_score_df.sort_values(by='score', ascending=False)
    #print("Mutual Information Scores:")
    #print(mi_score_df)
    # Select features with a score greater than the threshold
    sel_feature_list = mi_score_df[mi_score_df['score'] > threshold]['feature'].tolist()
    print(f"\nSelected {len(sel_feature_list)} features with MI score > {threshold}")
    print(sel_feature_list)
    return sel_feature_list
 
def forecast_at_submission_month(submission_month:int , df_ts_key:pd.DataFrame, horizon=None):
    if horizon is None:
        horizons = [0,1,2,3]
    training_data = create_features_and_labels_per_horizon(df_ts_key.query(f'fiscal_month <{submission_month}'),horizons=horizons)
    inference_data = create_features_and_labels_per_horizon(df_ts_key.query(f'fiscal_month=={submission_month}'),horizons=horizons)
    forecasts=[]
    for h in horizons:
        training_data_h = training_data.query(f'horizon=={h}').dropna(subset=['y'])
        inference_data_h = inference_data.query(f'horizon=={h}')
 
        if training_data_h.empty:
            print(training_data_h)
            continue
           
       
        if inference_data_h.empty:
            print(inference_data_h)
            continue
        training_data_h.replace([np.inf , -np.inf], np.nan,inplace=True)
        training_data_h.fillna(0,inplace=True)
 
        inference_data_h.replace([np.inf , -np.inf], np.nan,inplace=True)
        inference_data_h.fillna(0,inplace=True)
 
        features=['moving_avg_last_quarter',
         'lag_1_at_submission_month',
         'lag_2_at_submission_month',
         'lag_3_at_submission_month',
          'lag_4_at_submission_month',
           'lag_5_at_submission_month',
          'lag_6_at_submission_month',
          'lag_7_at_submission_month',
          'lag_8_at_submission_month',
            'ema_growth',
           'lag_9_at_submission_month',
           'lag_10_at_submission_month',
            'lag_11_at_submission_month',
            'lag_12_at_submission_month',
             'lag_12_last_year',
            'lag_11_last_year',
            'lag_10_last_year',
            'lag_9_last_year',
            'lag_8_last_year',
            'lag_7_last_year',
            'lag_6_last_year',
            'lag_5_last_year',
            'lag_4_last_year',
            'lag_3_last_year',
            'lag_2_last_year',
            'lag_1_last_year',
            'lag_1',
            'lag_2',
            'lag_3',
            'lag_4',
            'lag_5',
            'lag_6',
           'lag_7',
            'lag_8',
            'lag_9',
            'lag_10',
            'lag_11',
            'lag_12',
            'year',
            'month',
            'lag_1_moving_avg',
            'lag_2_moving_avg',
            'lag_3_moving_avg',
            'lag_4_moving_avg',
            'lag_5_moving_avg',
            'lag_6_moving_avg',
            'lag_7_moving_avg',
           'lag_8_moving_avg',
            'lag_9_moving_avg',
            'lag_10_moving_avg',
            'lag_11_moving_avg',
            'lag_12_moving_avg',
            'ema_5',
            'ema_10',
             'ema_20',
             'QoQ_percentage_growth',
             'sin_month',
            'cos_month',
            'YoY_growth',
             'Yoy_percentage_growth',
            'diff_at_submission_month',
            'moving_avg_1_months',
            'moving_avg_2_months',
            'moving_avg_3_months',
            'moving_avg_4_months',
            'moving_avg_5_months',
            'moving_avg_6_months',
            'moving_avg_7_months',
            'moving_avg_8_months',
            'moving_avg_9_months',
            'moving_avg_10_months',
            'moving_avg_11_months',
           'moving_avg_12_months',
            'moving_avg_13_months',
            'moving_avg_14_months',
            'moving_avg_15_months',
            'moving_avg_16_months',
            'moving_avg_17_months',
            'moving_avg_18_months',
            'moving_avg_19_months',
            'moving_avg_21_months',
            'moving_avg_22_months',
            'moving_avg_23_months',
            'moving_avg_24_months',
       
        'LOST_SALES_VALUE_USD','UNIT_PRICE','TOTAL_FOC_SALES_QTY']
 
        features=mi_feature_selection(df_ts_key,features,'SALES_VALUE_USD')
        features.append('month_of_year')
        model1 = create_pipeline_for_model_lgbm().fit(training_data_h[features],training_data_h['y'])
        forecast_quantity = model1.predict(inference_data_h[features])[0]
        forecasts.append(forecast_quantity)
   
 
    forecasted = pd.DataFrame()
    forecasted['forecast_sales'] = forecasts
    forecasted['submission_month'] = submission_month
    forecasted['horizon'] = horizons
 
    for col in ['ts_key','COUNTRY_CODE','MATERIAL_GRP4_CODE','MATERIAL_CODE','ATC_2_ID']:
        forecasted[col] = df_ts_key[col].iloc[0]
 
    future_months = generate_all_relevant_fiscal_months(submission_month, len(horizons))
 
    forecasted = pd.concat([forecasted, future_months],axis=1)
 
    return forecasted
       
 
 
 
 
 
 
df_in['fiscal_month'] = df_in.fiscal_month.astype('int64')
 
def replace_outliers(outliers: pd.Series, df_in: pd.DataFrame, input_col, output_col,submission_month):
    # Make a copy of the input DataFrame
    df_out = df_in.copy()
    # Initialize a new column to hold the clean data
    df_out['clipped_orders'] = df_out['SALES_VALUE_USD'].copy()
    # Filter past sales data based on submission month
    past_sales = df_out.query(f'fiscal_month < {submission_month}')['SALES_VALUE_USD']
    # Compute the mean value of past sales
    mean_value = past_sales.mean()
    # Get the indexes of outliers
    indexes = outliers.index[outliers].tolist()
    # Replace outlier values with the computed mean value
    df_out.iloc[indexes, df_out.columns.get_loc('clipped_orders')] = round(mean_value, 2)
    return df_out
 
 
 
def iqr_outlier_treat( df_in: pd.DataFrame,submission_month, input_col='SALES_VALUE_USD', output_col='clipped_orders') -> pd.DataFrame:
    """
     This method uses IQR  to identify outliers, returns a dataframe
    """
    submission_month=submission_month
    full_sales = df_in['SALES_VALUE_USD'].copy().reset_index(drop=True)
    past_sales = df_in.query(f'fiscal_month<{submission_month}')['SALES_VALUE_USD'].reset_index(drop=True)
    q1 = past_sales.quantile(q=0.25)
    q3 = past_sales.quantile(q=0.75)
    iqr = q3 - q1
    low_limit = q1 - 1.5 * iqr
    upper_limit = q3 + 1.5 * iqr
    outliers = (full_sales < low_limit) | (full_sales > upper_limit)
    df_out = replace_outliers(outliers, df_in, input_col, output_col,submission_month)
    return df_out
 
 
 
res= []
submission_month = 2023012
for code,data in df_in.groupby('ts_key'):
    print(code)
    if len(data.query(f'fiscal_month < {submission_month}')) < 30:
         continue
    #data=iqr_outlier_treat(data,submission_month)
    data_per_horizon = create_features_and_labels_per_horizon(data)
    forecast_partial  = forecast_at_submission_month(submission_month,data_per_horizon)
    res.append(forecast_partial)
 
df_forecasts = pd.concat(res,axis=0)
df_forecasts.drop(['start_month','end_month'],axis=1,inplace=True)
 
 
def evaluate_forecast(experiment_name: str, actuals_df:pd.DataFrame, forecasts_df:pd.DataFrame,mape_group_by=None, save_results=False):
    rolling_key=['ts_key','fiscal_month']
    monthly_mape_df = forecasts_df.merge(actuals_df,on=rolling_key,how='left', suffixes=('_x',None))
    #monthly_mape_df = monthly_mape_df.query('SALES_VALUE_USD !=0')
   
    monthly_mape_df['abs_error'] = np.abs(monthly_mape_df['SALES_VALUE_USD'] - monthly_mape_df['forecast_sales'])
    #monthly_mape_df['unit_price'] = monthly_mape_df['SALES_VALUE_USD'] / monthly_mape_df['NET_SALES_QTY']
    #monthly_mape_df['forecasted_sales_quantity'] = monthly_mape_df['unit_price'] * monthly_mape_df['forecast_quantity']
    print(monthly_mape_df.head(3))
 
 
    #monthly_mape_df.to_csv('rf_thailand_country_light_gbm_full_data_1.csv')
 
    if isinstance(monthly_mape_df, (pd.DataFrame,pd.Series)):
        if mape_group_by:
            result = monthly_mape_df.groupby('fiscal_month')['abs_error'].sum() / monthly_mape_df.groupby('fiscal_month')['SALES_VALUE_USD'].sum() * 100
        else:
            result = (np.sum(monthly_mape_df['abs_error']) / np.sum(monthly_mape_df['SALES_VALUE_USD']) *100).round(2)
       
        print(f"Final MAPE: {result}")        
        return result
 
    else:
        raise TypeError("monthly_mape_df has to be a dataframe")
 
    if save_results:
        monthly_mape_df.to_csv(f'{experiment_name}_forecasts.csv')
 
    return result,monthly_mape_df
 
 
evaluate_forecast('Malaysia_matgrp',df_in,df_forecasts,['fiscal_month','horizon'] , save_results=False)
 
def evaluate_forecast(experiment_name: str, actuals_df:pd.DataFrame, forecasts_df:pd.DataFrame,mape_group_by=None, save_results=False):
    rolling_key=['ts_key','fiscal_month']
    monthly_mape_df = forecasts_df.merge(actuals_df,on=rolling_key,how='left', suffixes=('_x',None))
    #monthly_mape_df = monthly_mape_df.query('SALES_VALUE_USD !=0')
   
    monthly_mape_df['abs_error'] = np.abs(monthly_mape_df['SALES_VALUE_USD'] - monthly_mape_df['forecast_sales'])
    #monthly_mape_df['unit_price'] = monthly_mape_df['SALES_VALUE_USD'] / monthly_mape_df['NET_SALES_QTY']
    #monthly_mape_df['forecasted_sales_quantity'] = monthly_mape_df['unit_price'] * monthly_mape_df['forecast_quantity']
    print(monthly_mape_df.head(3))
 
 
    #monthly_mape_df.to_csv('rf_thailand_country_light_gbm_full_data_1.csv')
 
    if isinstance(monthly_mape_df, (pd.DataFrame,pd.Series)):
        if mape_group_by:
            result = monthly_mape_df.groupby('fiscal_month')['abs_error'].sum() / monthly_mape_df.groupby('fiscal_month')['SALES_VALUE_USD'].sum() * 100
        else:
            result = (np.sum(monthly_mape_df['abs_error']) / np.sum(monthly_mape_df['SALES_VALUE_USD']) *100).round(2)
       
        print(f"Final MAPE: {result}")        
        return result
 
    else:
        raise TypeError("monthly_mape_df has to be a dataframe")
 
    if save_results:
        monthly_mape_df.to_csv(f'{experiment_name}_forecasts.csv')
 
    return result,monthly_mape_df
 
 

In [ ]:
def create_pipeline_for_model_lstm_new():
    categorical_columns = ['month_of_year']
    categories = [[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]]
    ordinal_encoder = OrdinalEncoder(categories=categories, handle_unknown='use_encoded_value', unknown_value=-1)
    model=Sequential([
        LSTM(units=10,return_sequences=True,input_shape=(1,1)),
        LSTM(units=5),
        Dense(units=1,activation='relu')
                        ])
    model.compile(optimizer=RMSprop(learning_rate=0.001),loss='mean_squared_error')
    pipeline=Pipeline([
        ('col_trans', ColumnTransformer(transformers=[('categorical', ordinal_encoder, categorical_columns)], remainder="passthrough")),
        ('scaler', StandardScaler()),
        ('lstm', model)
    ])
    return pipeline,model

def forecast_at_submission_month_new(submis... by Valli Raja Sekar
5:04 pm
Valli Raja Sekar
def forecast_at_submission_month_new(submission_month:int , df_ts_key:pd.DataFrame, horizon=None):
    if horizon is None:
        horizons = [0,1,2,3]
    training_data = create_features_and_labels_per_horizon(df_ts_key.query(f'fiscal_month <{submission_month}'),horizons=horizons)
    inference_data = create_features_and_labels_per_horizon(df_ts_key.query(f'fiscal_month=={submission_month}'),horizons=horizons)
    forecasts=[]
    for h in horizons:
        training_data_h = training_data.query(f'horizon=={h}').dropna(subset=['y'])
        inference_data_h = inference_data.query(f'horizon=={h}')
 
        if training_data_h.empty:
            print(training_data_h)
            continue

        if inference_data_h.empty:
            print(inference_data_h)
            continue
        training_data_h.replace([np.inf , -np.inf], np.nan,inplace=True)
        training_data_h.fillna(0,inplace=True)
 
        inference_data_h.replace([np.inf , -np.inf], np.nan,inplace=True)
        inference_data_h.fillna(0,inplace=True)
 
 
        #features=mi_feature_selection(df_ts_key,features,'SALES_VALUE_USD')
        #features.append('month_of_year')
        X_train,y_train=training_data_h[['y']],training_data_h['y']
        X_inference=inference_data_h[['y']]
 
        imputer = SimpleImputer(strategy='mean')
        scaler = StandardScaler()
        X_train_imputed = imputer.fit_transform(X_train)
        X_inference_imputed = imputer.transform(X_inference)
        X_train_scaled = scaler.fit_transform(X_train_imputed)
        X_inference_scaled = scaler.transform(X_inference_imputed)
        # Reshape for LSTM
        X_train_reshaped = np.reshape(X_train_scaled, (X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
        X_inference_reshaped = np.reshape(X_inference_scaled, (X_inference_scaled.shape[0], 1, X_inference_scaled.shape[1]))
        pipeline, model = create_pipeline_for_model_lstm_new()
        model.fit(X_train_reshaped, y_train, epochs=50, batch_size=32, verbose=0)
        lstm_forecasts_scaled = model.predict(X_inference_reshaped)
        # Inverse transform the scaled forecasts
        lstm_forecasts_scaled = scaler.inverse_transform(lstm_forecasts_scaled)
        forecast_quantity = lstm_forecasts_scaled.flatten()
        forecast_quantity = forecast_quantity[h]
        forecasts.append(forecast_quantity)

 
    forecasted = pd.DataFrame()
    forecasted['forecast_sales'] = forecasts
    forecasted['submission_month'] = submission_month
 
    for col in ['ts_key','REGION','MATERIAL_GRP4_CODE','MATERIAL_CODE']:
        forecasted[col] = df_ts_key[col].iloc[0]
 
    future_months = generate_all_relevant_fiscal_months(submission_month, len(horizons))
 
    forecasted = pd.concat([forecasted, future_months],axis=1)
 
    return forecasted

